## α-β-Pruning

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-heuristic.ipynb

In [ ]:
Cache = {}

def value(s, p, limit, alpha=-1, beta=1):
    global Cache
    if s in Cache:
        (val, a, b) = Cache[s]
        if a <= alpha and beta <= b:
            return val
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            val   = alphaBeta(s, p, limit, alpha, beta)
            Cache[s] = (val, alpha, beta)
            return val
    else:
        val = alphaBeta(s, p, limit, alpha, beta)
        Cache[s] = (val, alpha, beta)
        return val

In [ ]:
def alphaBeta(s, p, limit, alpha, beta):
    if finished(s, p):
        return utility(s, p)
    if limit == 0:
        return heuristic(s, p)
    val = alpha
    for ns in nextStates(s, p):
        val = max(val, -value(ns, opponent(p), limit-1, -beta, -alpha))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

Um den besten Schritt zu errechnen, den der gegebene Spieler `p` im State `s` ausführen kann, wurde die `bestMoveAlphaBeta()` Funktion implementiert. Diese nutzt die zuvor geschriebenen Funktionen `alphaBeta()` und `value()`.

Hierbei werden Schätzungen für jeden möglichen Zug erstellt und danach der beste Zug ausgewählt.

In [ ]:
def bestMoveAlphaBeta(s, p):
    # Clear cache
    global Cache
    Cache = {}

    moves = [
        (-value(s, opponent(p), 2), s)
        for s in nextStates(s, p)
    ]
    return max(moves, key=lambda m: m[0])